In [1]:
using Pkg

In [12]:
ENV["CPLEX_STUDIO_BINARIES"] = "C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio_Community129\\cplex\\bin\\x64_win64"

"C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio_Community129\\cplex\\bin\\x64_win64"

In [13]:
Pkg.build("CPLEX")

   Building MbedTLS → `C:\Users\Geoffroy Leconte\.julia\packages\MbedTLS\a1JFn\deps\build.log`
   Building CPLEX ──→ `C:\Users\Geoffroy Leconte\.julia\packages\CPLEX\Hxpuk\deps\build.log`


In [5]:
using CPLEX
using QPSReader
using QuadraticModels
using NLPModels
using SolverTools
using SolverBenchmark
using DataFrames

In [29]:
function createQuadraticModel(qpdata)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0)

end

function display_results_CPlex(xpress_model)
    # show results
    sol = CPLEX.get_solution(model_cplex)
    println("soln = $(sol)")

    objv = CPLEX.get_objval(model_cplex)
    println("objv = $(objv)")
end
    

function OptimizeCPlex(QM)
    SM = SlackModel(QM)
    
    env = CPLEX.Env()
    
    model = CPLEX.cplex_model(env; f = grad(SM, zeros(length(SM.meta.x0))),
                Aeq = jac(SM, SM.meta.x0), beq = SM.meta.lcon, 
                lb = SM.meta.lvar, ub = SM.meta.uvar)
    # H = ... if quadratic problem
     # run optimization
    CPLEX.optimize!(model)
 
    println(CPLEX.c_api_solninfo(model))
    
    if model.terminator == [0]
        status = :acceptable
    else
        status = :unknown
    end
    
    stats = GenericExecutionStats(status, SM, solution = CPLEX.get_solution(model),
                                  objective = CPLEX.get_objval(model), iter = Int64(CPLEX.c_api_getitcnt(model)),
                                  multipliers = CPLEX.get_constr_duals(model))
    
    return stats
end

function OptimizeCPlex(qpdata::QPSData)
    return OptimizeCPlex(createQuadraticModel(qpdata))
end

OptimizeCPlex (generic function with 2 methods)

In [30]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [31]:
stats2 = OptimizeCPlex(qpdata2)
println(stats2)

(2, 1, 1, 1)
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -464.7531428571428
  primal feasibility: Inf
  dual feasibility: Inf
  solution: [80.0  25.5  54.5  84.80000000000001 ⋯ 19.30714285714286]
  multipliers: [-0.6285714285714286  0.0  -0.34477142857142856  -0.2285714285714286 ⋯ -0.0]
  iterations: 7
  elapsed time: Inf
